In [15]:
import pandas as pd 
LAYER=32

In [16]:
languages = ['cpp','cs','java','js','php','python','sh','ts']

In [25]:
def get_label_df(lang):
    source_lang_path = f'data/ds67{lang}/output2/LFCLF_embedding_human_eval_deepseek-ai_deepseek-coder-6.7b-instruct_{LAYER}.parquet'
    label_lang_path = f'data/ds67{lang}/output2/LFCLF_embedding_human_eval_deepseek-ai_deepseek-coder-6.7b-instruct_1_label.parquet'
    # print(source_lang_path)
    # print(label_lang_path)
    df_source = pd.read_parquet(source_lang_path)
    df_label = pd.read_parquet(label_lang_path)
    # print(df_label.head(1))
    result = list()
    for idx,row in df_source.iterrows():
        row['label'] = 1 if df_label.at[idx, 'label'] else 0
        result.append(row)
    return pd.DataFrame(result)    

In [28]:
SETTING = 'cross_language'
out_dir = 'data/data_for_classify'
import os

for language in languages:
    test_df = get_label_df(language)
    trains = list()
    for otherlang in languages:
        if otherlang == language:
            continue
        trains.append(get_label_df(otherlang))
    train_df = pd.concat(trains)
    train_df = train_df.sample(frac = 1)
    out_dir_lang = os.path.join(out_dir,language)
    if not os.path.exists(out_dir_lang):
        os.mkdir(out_dir_lang)
    test_df.to_parquet(os.path.join(out_dir_lang,'test.parquet'))
    train_df.to_parquet(os.path.join(out_dir_lang,'train.parquet'))
    print(language,test_df.shape,train_df.shape)
    print(test_df['label'].value_counts())
    print(train_df['label'].value_counts())


cpp (1610, 11) (11186, 11)
label
0    810
1    800
Name: count, dtype: int64
label
1    6213
0    4973
Name: count, dtype: int64
cs (1577, 11) (11219, 11)
label
0    853
1    724
Name: count, dtype: int64
label
1    6289
0    4930
Name: count, dtype: int64
java (1579, 11) (11217, 11)
label
1    1010
0     569
Name: count, dtype: int64
label
1    6003
0    5214
Name: count, dtype: int64
js (1610, 11) (11186, 11)
label
1    1046
0     564
Name: count, dtype: int64
label
1    5967
0    5219
Name: count, dtype: int64
php (1610, 11) (11186, 11)
label
1    817
0    793
Name: count, dtype: int64
label
1    6196
0    4990
Name: count, dtype: int64
python (1640, 11) (11156, 11)
label
1    1279
0     361
Name: count, dtype: int64
label
1    5734
0    5422
Name: count, dtype: int64
sh (1580, 11) (11216, 11)
label
0    1100
1     480
Name: count, dtype: int64
label
1    6533
0    4683
Name: count, dtype: int64
ts (1590, 11) (11206, 11)
label
1    857
0    733
Name: count, dtype: int64
label
1    6

In [30]:
train_df.head(1)

,task_id,completion_id,num_tokens,generation,first_token_embedding,last_token_embedding,first_token_code_embedding,last_token_code_embedding,has_error,extracted_code,label
314,Python/31,Python/31_4,173,"```python\ndef is_prime(n):\n """"""Return tru...","[0.1396484375, 0.310546875, 0.043701171875, 0....","[0.1259765625, 0.50390625, 0.09228515625, 0.14...","[0.2451171875, 0.2060546875, 0.140625, 0.10302...","[0.2041015625, 0.1611328125, -0.06689453125, 0...",False,"\ndef is_prime(n):\n """"""Return true if a gi...",1


In [31]:
SETTING = 'cross_language'
out_dir = 'data/data_for_classify'
import os
import re
train = list()
test = list()
dfs = list()
for language in languages:
    dfs.append(get_label_df(language))
data_df = pd.concat(dfs)

with open('data/train.txt') as f:
    train_id = [int(l.strip()) for l in f.readlines()]
with open('data/test.txt') as f:
    test_id = [int(l.strip()) for l in f.readlines()]

for _,row in data_df.iterrows():
    task_id = row['task_id']
    numbers = re.findall(r'\d+', task_id)
    if len(numbers) <= 0:
        train.append(row)
        continue
    if int(numbers[0]) in train_id:
        train.append(row)
    else:
        test.append(row)

train_df = pd.DataFrame(train)
test_df = pd.DataFrame(test)

train_df = train_df.sample(frac = 1)
out_dir_lang = os.path.join(out_dir,'cross_task')
if not os.path.exists(out_dir_lang):
    os.mkdir(out_dir_lang)
test_df.to_parquet(os.path.join(out_dir_lang,'test.parquet'))
train_df.to_parquet(os.path.join(out_dir_lang,'train.parquet'))
print(language,test_df.shape,train_df.shape)
print(test_df['label'].value_counts())
print(train_df['label'].value_counts())


ts (2490, 11) (10306, 11)
label
1    1347
0    1143
Name: count, dtype: int64
label
1    5666
0    4640
Name: count, dtype: int64


In [32]:
train_df.head(10)

,task_id,completion_id,num_tokens,generation,first_token_embedding,last_token_embedding,first_token_code_embedding,last_token_code_embedding,has_error,extracted_code,label
1373,Python/137,Python/137_3,351,Here is the completed function:\n\n```python\n...,"[-0.1845703125, 0.298828125, -0.359375, 0.1040...","[0.236328125, 0.380859375, 0.12890625, 0.28320...","[0.3984375, 0.171875, 0.322265625, 0.177734375...","[-0.049560546875, 0.25390625, -0.0859375, 0.07...",True,"\ndef compare_one(a, b):\n if isinstance(a,...",0
656,HumanEval_117_select_words,HumanEval_117_select_words_6,382,Here is the completed function in C++:\n\n```c...,"[-0.2431640625, 0.345703125, -0.373046875, 0.1...","[0.2021484375, 0.341796875, 0.08349609375, 0.3...","[0.28125, 0.248046875, -0.059326171875, 0.1157...","[0.333984375, 0.2431640625, -0.10791015625, 0....",False,\n\nstd::vector<std::string> select_words(std:...,1
211,HumanEval_74_total_match,HumanEval_74_total_match_1,175,Here is the completed function in JavaScript:\...,"[-0.1796875, 0.33984375, -0.349609375, 0.11474...","[0.2099609375, 0.36328125, 0.095703125, 0.3242...","[0.224609375, 0.318359375, 0.181640625, 0.0664...","[0.318359375, 0.203125, -0.1650390625, 0.36328...",False,"\nfunction total_match(lst1, lst2){\n let t...",1
697,HumanEval_92_any_int,HumanEval_92_any_int_7,431,Here is the completed function in Java:\n\n```...,"[-0.25, 0.28125, -0.361328125, 0.1494140625, 0...","[0.158203125, 0.361328125, 0.04345703125, 0.35...","[0.126953125, 0.1025390625, 0.04150390625, 0.1...","[0.177734375, 0.21875, -0.0216064453125, -0.00...",False,"\n public static boolean anyInt(float x, fl...",0
636,HumanEval_117_select_words,HumanEval_117_select_words_6,329,Here is the completed function in TypeScript:\...,"[-0.18359375, 0.357421875, -0.36328125, 0.1181...","[0.19140625, 0.38671875, 0.053466796875, 0.335...","[0.419921875, 0.31640625, 0.197265625, 0.13671...","[0.380859375, 0.2578125, -0.224609375, 0.17871...",False,"\nfunction select_words(s: string, n: number):...",0
1294,HumanEval_45_triangle_area,HumanEval_45_triangle_area_4,139,Here is the completed function in JavaScript:\...,"[-0.25, 0.296875, -0.373046875, 0.0966796875, ...","[0.189453125, 0.37109375, 0.07666015625, 0.245...","[0.19921875, 0.010009765625, 0.255859375, 0.23...","[0.205078125, 0.10107421875, -0.1748046875, 0....",False,"\nfunction triangle_area(a, h){\n return 0....",1
16,HumanEval_89_encrypt,HumanEval_89_encrypt_6,385,Here is the completed function:\n\n```java\nim...,"[-0.220703125, 0.328125, -0.36328125, 0.125, 0...","[0.232421875, 0.4453125, 0.1279296875, 0.41015...","[0.349609375, 0.1865234375, 0.08203125, 0.1542...","[0.146484375, 0.322265625, -0.036865234375, -0...",False,\n public static String encrypt(String s) {...,1
569,HumanEval_0_has_close_elements,HumanEval_0_has_close_elements_9,263,Here is the completed function in bash:\n\n```...,"[-0.205078125, 0.341796875, -0.40234375, 0.060...","[0.1728515625, 0.443359375, 0.042236328125, 0....","[0.30859375, 0.287109375, 0.119140625, 0.03735...","[0.28515625, 0.298828125, -0.12109375, 0.16601...",False,\n\nhas_close_elements() {\n IFS=' ' read -...,0
836,HumanEval_47_median,HumanEval_47_median_6,219,Here is the completed function:\n\n```java\nim...,"[-0.28125, 0.3359375, -0.365234375, 0.13867187...","[0.173828125, 0.314453125, 0.07861328125, 0.34...","[0.349609375, 0.1484375, 0.07568359375, 0.1845...","[0.3671875, 0.1591796875, -0.0654296875, -0.00...",False,\n public static float median(ArrayList<Lon...,1
787,Python/78,Python/78_7,134,Here is the completed function:\n\n```python\n...,"[-0.1982421875, 0.298828125, -0.373046875, 0.1...","[0.1845703125, 0.416015625, 0.1279296875, 0.25...","[0.271484375, 0.126953125, 0.419921875, -0.031...","[0.4375, 0.041259765625, 0.034912109375, 0.086...",False,"\ndef hex_key(num):\n primes = {'2', '3', '...",1
